In [1]:
import json
import re
from backend.src.FastTextRank4Word import FastTextRank4Word
from backend.src.FastTextRank4Sentence import FastTextRank4Sentence
from backend.src.utils import MySQLAgent
import datetime

Building prefix dict from c:\Users\sean.chang\yfy\git\credit_investigation\backend\dict\dict.big.txt ...
Loading model from cache C:\Users\SEAN~1.CHA\AppData\Local\Temp\jieba.u7daa19b2c9e6a83f2c21b3bf514aa122.cache
Loading model cost 0.849 seconds.
Prefix dict has been built successfully.


In [2]:
def read_config(path):
    try:
        with open(path, 'r') as file:
            configs = json.load(file)

        return configs
    except FileNotFoundError:
        print(f"The file {path} was not found.")
    except json.JSONDecodeError:    
        print(f"Error decoding JSON from the file {path}.")

configs = read_config(path="./backend/conn/connections.json")
job_configs = configs["CREDITREPORT"]['Crawler_mysql_conn_info']
sql_agent = MySQLAgent(job_configs)

In [3]:
query = """
SELECT judgement_date, querykey, subjectkey, judgement_text01
FROM judgement_result
WHERE business_accounting_no = 83387850 AND judgement_date is not null
"""
df_judgement = sql_agent.read_table(query=query)

In [4]:
content_string = df_judgement.iloc[0]['judgement_text01']

In [5]:
def clean_data(data):
    # Remove specific unicode characters
    cleaned_data = re.sub(r'\u3000', '', data)
    
    # Replace multiple spaces with a single space
    cleaned_data = re.sub(r'\s{2,}', ' ', cleaned_data)
    
    # Remove leading and trailing spaces on each line
    cleaned_data = re.sub(r'^\s+|\s+$', '', cleaned_data, flags=re.MULTILINE)
    
    # Remove redundant newlines
    cleaned_data = re.sub(r'\n\s*\n', '\n', cleaned_data)

    cleaned_data = re.sub(r'\n', '', cleaned_data)

    cleaned_data = re.sub(r'\s', '', cleaned_data)
    
    return cleaned_data

def extract_section(data, start_keyword, end_keyword):
    # Use regex to find the text between start_keyword and end_keyword
    pattern = re.compile(f'{start_keyword}(.*?){end_keyword}', re.DOTALL)
    match = pattern.search(data)
    if match:
        return match.group(1).strip()
    else:
        return None

In [6]:
clean_text = clean_data(df_judgement.iloc[0].judgement_text01)
clean_text

'臺灣高等法院高雄分院刑事判決九十年度上易字第六０六號上訴人即被告丙○○丁○○右一人選任辯護人林樹根律師邱麗妃律師莊雯琇律師上訴人即被告甲○○選任辯護人康清敬右上訴人因偽造文書等案件，不服臺灣高雄地方法院八十八年度易字第一一三二號中華民國八十九年十二月十四日第一審判決（起訴案號：臺灣高雄地方法院檢察署八十七年度偵字第一四六０四號），提起上訴，本院判決如左：主文上訴駁回。事實一、丁○○曾於民國八十六年間，因侵占案件，經法院判處有期徒刑五月確定，並於八十六年十月二十四日易科罰金執行完畢，仍不知悔改，其係皮孩皮飾有限公司（下稱皮孩公司）及漾格服飾有限公司（下稱漾格公司）之實際負責人，其配偶丙○○於八十五年三月間積欠乙○○○會款及借款共新台幣（下同）五十四萬元，並於八十六年六月十九日經最高法院判決應給付乙○○○五十四萬元確定在案，係乙○○○之債務人。乙○○○為強制執行上開債權，乃於八十七年二月十六日就丙○○於第三人即三芳化學工業股份有限公司之退休金一百四十一萬七千八百二十一元向臺灣高雄地方院民事執行處聲請強制執行（臺灣高雄地方院八十六年執字第二六七四四號、八十五年度執全字第二五０七號），嗣丙○○為阻擾乙○○○於前述民事強制執行程序中受償，並為隱匿自己之財產，遂與其夫丁○○及友人甲○○，共同基於明知為不實事項而使公務員登載不實及意圖損害債權人乙○○○之債權之概括犯意聯絡，明知丙○○與甲○○並無票據債權債務關係，三人竟於八十七年一月間，在甲○○位於高雄市○○區○○路店中，由甲○○填載丙○○為發票人，而由丙○○本人蓋章，簽發發票日分別為八十四年三月十六日、同年五月十六日、同年六月十六日、同年六月二十日、同年三月九日，到期日分別為八十四年四月十六日、同年六月十六日、同年七月十六日、同年七月二十日、同年四月九日，金額分別為一百萬元四張、一百三十萬元（金額共五百三十萬元），票號分別為ＴＨ0000000、ＴＨ0000000、ＴＨ0000000、ＴＨ0000000、ＴＨ0000000之本票五紙（下稱系爭本票），交由甲○○持向臺灣高雄地方法院聲請裁定准許本票強制執行，致使臺灣高雄地方法院承辦是項公務之法官，僅經由形式審查，於八十七年二月二日在其辦公處所內，將丙○○積欠甲○○票據票款五百三十萬元之不實事項，登載於職務上所掌之民事裁定（臺灣高雄地方法院八十七年票字第一二九五號准許強制執行裁定）公

In [12]:
with open("clean_text.txt", "w") as file:
    # Write the string to the file
    file.write(clean_text)

In [7]:
model = FastTextRank4Word(tol=0.0001, window=2)
old_time = datetime.datetime.now()
print('關鍵詞彙:')
old_time = datetime.datetime.now()
po=model.summarize(clean_text, 20)
print(po)
# print(datetime.datetime.now() - old_time)

關鍵詞彙:
['被告', '本票', '分配', '公司', '刑法', '地方法院', '上開', '六月', '同年', '行使', '有期徒刑', '本院', '年度', '裁定', '支票', '部分', '易科', '犯意', '退休金', '皮孩']


In [8]:
mod = FastTextRank4Sentence(use_w2v=False,tol=0.0001)
old_time = datetime.datetime.now()
print('摘要:')
old_time = datetime.datetime.now()
po=mod.summarize(clean_text, 2)
print(po)
# print(datetime.datetime.now() - old_time)

摘要:
['乙○○○為強制執行上開債權，乃於八十七年二月十六日就丙○○於第三人即三芳化學工業股份有限公司之退休金一百四十一萬七千八百二十一元向臺灣高雄地方院民事執行處聲請強制執行（臺灣高雄地方院八十六年執字第二六七四四號、八十五年度執全字第二五０七號），嗣丙○○為阻擾乙○○○於前述民事強制執行程序中受償，並為隱匿自己之財產，遂與其夫丁○○及友人甲○○，共同基於明知為不實事項而使公務員登載不實及意圖損害債權人乙○○○之債權之概括犯意聯絡，明知丙○○與甲○○並無票據債權債務關係，三人竟於八十七年一月間，在甲○○位於高雄市○○區○○路店中，由甲○○填載丙○○為發票人，而由丙○○本人蓋章，簽發發票日分別為八十四年三月十六日、同年五月十六日、同年六月十六日、同年六月二十日、同年三月九日，到期日分別為八十四年四月十六日、同年六月十六日、同年七月十六日、同年七月二十日、同年四月九日，金額分別為一百萬元四張、一百三十萬元（金額共五百三十萬元），票號分別為ＴＨ0000000、ＴＨ0000000、ＴＨ0000000、ＴＨ0000000、ＴＨ0000000之本票五紙（下稱系爭本票），交由甲○○持向臺灣高雄地方法院聲請裁定准許本票強制執行，致使臺灣高雄地方法院承辦是項公務之法官，僅經由形式審查，於八十七年二月二日在其辦公處所內，將丙○○積欠甲○○票據票款五百三十萬元之不實事項，登載於職務上所掌之民事裁定（臺灣高雄地方法院八十七年票字第一二九五號准許強制執行裁定）公文書上，足生損害於債權人乙○○○及臺灣高雄地方法院非訟事件之正確性。', '理由一、訊據上訴人即被告丙○○、丁○○、甲○○均矢口否認有何右揭犯行，辯稱：被告丁○○於八十二年起，即多次向被告甲○○借款做經營皮孩公司及漾格公司之用，嗣於八十三年八月間，改由其妻即被告丙○○持皮孩公司支票，並由被告丙○○在其上背書，陸續再向被告甲○○借款五百三十萬元，且同時由被告丙○○在皮孩公司另開立未載發票日，到期日八十四年三月九日、八十四年四月十六日、八十四年五月十六日、八十四年六月十六日、八十四年七月十六日票面金額依序一百三十萬元、一百萬元、一百萬元、一百萬元、一百萬元之本票五張上背書（下稱背書本票），作為上開支票之擔保，後因上開支票無法兌現，而作為擔保之上開背書本票，亦因未載發票日無法行使票據權利，被告三人遂於八十七年一月間，在被告甲○○上開店中

In [9]:
len(po)

2

In [10]:
summary1 = '''乙○○○為強制執行上開債權，乃於八十七年二月十六日就丙○○於第三人即三芳化學工業股份有限公司之退休金一百四十一萬七千八百二十一元向臺灣高雄地方院民事執行處聲請強制執行（臺灣高雄地方院八十六年執字第二六七四四號、八十五年度執全字第二五０七號），嗣丙○○為阻擾乙○○○於前述民事強制執行程序中受償，並為隱匿自己之財產，遂與其夫丁○○及友人甲○○，共同基於明知為不實事項而使公務員登載不實及意圖損害債權人乙○○○之債權之概括犯意聯絡，明知丙○○與甲○○並無票據債權債務關係，三人竟於八十七年一月間，在甲○○位於高雄市○○區○○路店中，由甲○○填載丙○○為發票人，而由丙○○本人蓋章，簽發發票日分別為八十四年三月十六日、同年五月十六日、同年六月十六日、同年六月二十日、同年三月九日，到期日分別為八十四年四月十六日、同年六月十六日、同年七月十六日、同年七月二十日、同年四月九日，金額分別為一百萬元四張、一百三十萬元（金額共五百三十萬元），票號分別為ＴＨ0000000、ＴＨ0000000、ＴＨ0000000、ＴＨ0000000、ＴＨ0000000之本票五紙（下稱系爭本票），交由甲○○持向臺灣高雄地方法院聲請裁定准許本票強制執行，致使臺灣高雄地方法院承辦是項公務之法官，僅經由形式審查，於八十七年二月二日在其辦公處所內，將丙○○積欠甲○○票據票款五百三十萬元之不實事項，登載於職務上所掌之民事裁定（臺灣高雄地方法院八十七年票字第一二九五號准許強制執行裁定）公文書上，足生損害於債權人乙○○○及臺灣高雄地方法院非訟事件之正確性。理由一、訊據上訴人即被告丙○○、丁○○、甲○○均矢口否認有何右揭犯行，辯稱：被告丁○○於八十二年起，即多次向被告甲○○借款做經營皮孩公司及漾格公司之用，嗣於八十三年八月間，改由其妻即被告丙○○持皮孩公司支票，並由被告丙○○在其上背書，陸續再向被告甲○○借款五百三十萬元，且同時由被告丙○○在皮孩公司另開立未載發票日，到期日八十四年三月九日、八十四年四月十六日、八十四年五月十六日、八十四年六月十六日、八十四年七月十六日票面金額依序一百三十萬元、一百萬元、一百萬元、一百萬元、一百萬元之本票五張上背書（下稱背書本票），作為上開支票之擔保，後因上開支票無法兌現，而作為擔保之上開背書本票，亦因未載發票日無法行使票據權利，被告三人遂於八十七年一月間，在被告甲○○上開店中，由丙○○簽發其本人為發票人之前揭系爭本票五紙換回上開支票及背書本票，並非假造債權云云。核被告三人明知上開本票債權為不實事項，而使公務員登載於職務上所掌之民事裁定公文書，足以生損害於真正債權人及法院對非訟事件之正確性，嗣被告三人推由被告甲○○持前揭准許強制執行裁定聲請強制執行，而加以行使，應成立刑法第二百十四條、第二百十六條之行使使公務員登載不實文書罪；嗣經執行法院准予合併執行，使執行法院再將此不實之本票債權登載於分配表上，亦足生損害於執行法院對參與分配審核之正確性及真正債權人分配債權之成數，且被告三人係在強制執行程序終結前為之，其參與分配之目的，在減少真正債權人之分配成數而隱匿財產，顯有損害債權人債權之意圖，是應成立刑法第二百十四條之使公務員登載不實罪及刑法第三百五十六條之損害債權罪。再告訴人與被告丙○○之上開債權債務，業經最高法院於八十六年六月十九日判決確定，此有判決書一份在卷可按，是告訴人自是時起，即取得強制執行名義，被告三人明知被告甲○○、丙○○間，並無上開五百三十萬元之債權債務關係，竟於告訴人取得強制執行名義後，強制執行程序尚未終結前之八十七年一月二十日具狀聲請系爭本票裁定強制執行，進而參與上開退休金之分配，堪認被告等人於八十七年一月間始共同決意簽發系爭本票五紙。'''

summary2 = '''乙○○○為強制執行上開債權，乃於八十七年二月十六日就丙○○於第三人即三芳化學工業股份有限公司之退休金一百四十一萬七千八百二十一元向臺灣高雄地方院民事執行處聲請強制執行（臺灣高雄地方院八十六年執字第二六七四四號、八十五年度執全字第二五０七號），嗣丙○○為阻擾乙○○○於前述民事強制執行程序中受償，並為隱匿自己之財產，遂與其夫丁○○及友人甲○○，共同基於明知為不實事項而使公務員登載不實及意圖損害債權人乙○○○之債權之概括犯意聯絡，明知丙○○與甲○○並無票據債權債務關係，三人竟於八十七年一月間，在甲○○位於高雄市○○區○○路店中，由甲○○填載丙○○為發票人，而由丙○○本人蓋章，簽發發票日分別為八十四年三月十六日、同年五月十六日、同年六月十六日、同年六月二十日、同年三月九日，到期日分別為八十四年四月十六日、同年六月十六日、同年七月十六日、同年七月二十日、同年四月九日，金額分別為一百萬元四張、一百三十萬元（金額共五百三十萬元），票號分別為ＴＨ0000000、ＴＨ0000000、ＴＨ0000000、ＴＨ0000000、ＴＨ0000000之本票五紙（下稱系爭本票），交由甲○○持向臺灣高雄地方法院聲請裁定准許本票強制執行，致使臺灣高雄地方法院承辦是項公務之法官，僅經由形式審查，於八十七年二月二日在其辦公處所內，將丙○○積欠甲○○票據票款五百三十萬元之不實事項，登載於職務上所掌之民事裁定（臺灣高雄地方法院八十七年票字第一二九五號准許強制執行裁定）公文書上，足生損害於債權人乙○○○及臺灣高雄地方法院非訟事件之正確性。理由一、訊據上訴人即被告丙○○、丁○○、甲○○均矢口否認有何右揭犯行，辯稱：被告丁○○於八十二年起，即多次向被告甲○○借款做經營皮孩公司及漾格公司之用，嗣於八十三年八月間，改由其妻即被告丙○○持皮孩公司支票，並由被告丙○○在其上背書，陸續再向被告甲○○借款五百三十萬元，且同時由被告丙○○在皮孩公司另開立未載發票日，到期日八十四年三月九日、八十四年四月十六日、八十四年五月十六日、八十四年六月十六日、八十四年七月十六日票面金額依序一百三十萬元、一百萬元、一百萬元、一百萬元、一百萬元之本票五張上背書（下稱背書本票），作為上開支票之擔保，後因上開支票無法兌現，而作為擔保之上開背書本票，亦因未載發票日無法行使票據權利，被告三人遂於八十七年一月間，在被告甲○○上開店中，由丙○○簽發其本人為發票人之前揭系爭本票五紙換回上開支票及背書本票，並非假造債權云云。'''

word_count_summary1 = len(summary1)
word_count_summary2 = len(summary2)

print(f"摘要1的字數: {word_count_summary1}")
print(f"摘要2的字數: {word_count_summary2}")


摘要1的字數: 1535
摘要2的字數: 1034


In [11]:
Prompt_candidate = """你現在要以一個專業的繁體中文法院判決書閱讀整理助手來協助我針對我清理過的法院判決做整理。

重新整理以下我提供摘要，配合我找出的10個關鍵字，將其改寫成語意通順的摘要，並滿足以下條件

1. 這份判決整理只列出投資人審核該公司在意的內容，如公司名稱三芳化學
2. 文長不超過500字
"""